# Virtual ADC example
---


## Load the bitstream into the Programmable Logic

In [ ]:
%reload_ext autoreload
%autoreload 2
# Import the X-HEEP Python class
from pynq import x_heep
# Allow the import of all modules in this directory
import sys
import os

SDK_PATH = "/home/xilinx/x-heep-femu-sdk/"
RISCV_PATH = 'sw/riscv/'
APP_PATH = SDK_PATH + RISCV_PATH + 'apps/adc-emu/'
ADC_BIN_NAME = 'adc_in'

sys.path.append(SDK_PATH + "sw/arm/apps/adc-emu/src")

from timeseries import *
from report import *

# Load the X-HEEP bitstream
xh = x_heep()

## Generate an input array to use as signal 

In [ ]:
import numpy as np
from timeseries import Timeseries

INPUT_DATA_LENGTH           = 1550

# Take a pre-generated binary file
src         = SDK_PATH + 'sw/arm/apps/adc-emu/out/'
src_name    = 'Raw_Norm_pMean.sub_Norm'
ext_bin     = '.bin'
ext_pkl     = '.pkl'
dst         = APP_PATH
dst_name    = ADC_BIN_NAME

# Load the data as a timeseries
din = Timeseries.load(src+src_name+ext_pkl)
# Turn all values into unsigned by offsetting them
din.data  += abs(min(din.data))

# To abide by the memory constraint, downsample the signal
# INPUT_DATA_LENGTH = len(din.data) # Remove downsampling
inital_samples = len(din.data)
downsample = int(len(din.data)/INPUT_DATA_LENGTH)
din.data = din.data[::downsample]
din.time = din.time[::downsample]

# Export the binary file to the application folder
din.export_bin(path=APP_PATH, name=ADC_BIN_NAME)

INPUT_DATA_LENGTH = len(din.data)
print(f"Sent {INPUT_DATA_LENGTH}/{inital_samples} words as input")

In [ ]:
# din.data[0:5]  + OFFSET

## Compile the code and run the application
You can change the variable `SKIP_COMPILE`  to 1 to avoid compiling each time.

You can change the variable `SKIP_MEMORY`   to 1 to avoid generating the ADC memory each time.

You can change the variable `VERBOSE`       to 0 to supress the live printing of the UART.

In [ ]:
import time
import math
# Skip compiling if all you want to do is run the application
# Note that the compilation is affected by the parameters written to the definitions.h file
SKIP_COMPILE    = 0
# Skip the memory generation
SKIP_MEMORY     = 0
# Wether to see the X-HEEP UART output live.
VERBOSE         = 0
# Enable the printfs inside X-HEEP application. This is not the same as VERBOSE, as it actually reflects in C-code and therefore in performance.
ENABLE_PRINTF   = 1

# Define some standards to parse the X-HEEP output
OUTPUT_START_SEQ    = "# "
OUTPUT_DIVIDER      = "|"
OUTPUT_END_SEQ      = "#---"   

# Write the definitions header file to share information with the C code
defs = APP_PATH + 'definitions.h'
xh.open_definitions (defs, "DEFINITIONS")
xh.add_definition   (defs, "INPUT_DATA_LENGTH", INPUT_DATA_LENGTH)
xh.add_definition   (defs, "ENABLE_PRINTF",     ENABLE_PRINTF)
xh.add_definition   (defs, "OUTPUT_START_SEQ",  OUTPUT_START_SEQ)
xh.add_definition   (defs, "OUTPUT_DIVIDER",    OUTPUT_DIVIDER)
xh.add_definition   (defs, "OUTPUT_END_SEQ",    OUTPUT_END_SEQ)
xh.close_definitions (defs)

# Compile the application
xh.compile_app("adc-emu")

# Start a background thread that reads from the Virtual ADC
data_file_name = "/home/xilinx/x-heep-femu-sdk/sw/riscv/build/adc_in.bin"
ddr_thread, ddr_sf = xh.virtual_adc_thread_start(data_file_name)

# Start a background thread that reads from the UART
t, sf = xh.thread_start(VERBOSE)

# Set the performance counters
perf_cnt = xh.init_perf_cnt()       # Init the performance counters
xh.reset_perf_cnt(perf_cnt)         # Reset the performance counters
xh.start_perf_cnt_manual(perf_cnt)  # Start the performance counters

time.sleep(3)

out, _ = xh.run_app()               # Run the application (disregarding the error output)

# Finish
xh.stop_perf_cnt(perf_cnt)          # Stop the performance counters
xh.thread_stop(t, sf)               # Stop the background thread that reads from the UART
xh.thread_stop(ddr_thread, ddr_sf)  # Stop the ddr circular buffer controller


In [ ]:
# adc = xh.init_adc_mem()
# xh.reset_adc_mem(adc)
# adc.read(0x00)
# virtual_adc, ddr_buff = xh.init_virtual_adc(adc_size)
# xh.start_virtual_adc(virtual_adc, ddr_buff, "/home/xilinx/x-heep-femu-sdk/sw/riscv/build/adc_in.bin", INPUT_DATA_LENGTH)


In [ ]:
# adc.write(0*4, 2)
# adc.read(0*4) >> 24

In [ ]:
# file = open("/home/xilinx/x-heep-femu-sdk/sw/riscv/build/adc_in.bin", mode="rb")
# file_byte = file.read()
# print(file_byte[0])
# file.close()

In [ ]:
xh.uart_data

In [ ]:
xh.uart_data[-1]

### Parse the UART output

In [ ]:
# Extract the UART output from X-HEEP
app_out = xh.uart_data

# Find positions of strings starting with "#"
starts = [i for i, s in enumerate(app_out) if s.startswith(OUTPUT_START_SEQ)]
# Find positions of strings starting with "#"
ends = [i for i, s in enumerate(app_out) if s.startswith(OUTPUT_END_SEQ)]

# Sanity check to make sure that all output lists are closed
if len(starts) != len(ends): sys.exit("❌ Invalid output")

# Will divide the outputs in a list of dictionaries containing headers and data
outs = []
for seq in range(len(starts)):
    s = starts[seq]
    e = ends[seq]
    # Strip the headers of the format markers
    headers = app_out[s].split(OUTPUT_DIVIDER)
    headers = [ h.strip().replace(OUTPUT_START_SEQ,'') for h in headers ]
    # Strip the data of the format markers and covert it into a numpy array of integers
    data  = [line.split(OUTPUT_DIVIDER) for line in app_out[s+1:e]]
    data = np.asarray([ [int(i.strip()) for i in d] for d in data ]).T
    # Add the result to the list of dictionaries
    outs.append( {'headers':headers, 'data':data } )

# Convert all outputs to timeseries
# Each timeseries will be part of a sequence: a group of timeseries of the same length (were printed as different columns of a same process)
# All timeseries of a same sequence share timescale (that could be none)
raw_series = []
for seq in outs:
    seqs = []
    cols_i = 0 
    cols_f = len(seq['headers'])
    time = []
    # If some first column of a sequence is a a timestamp array, use it as time of all sequences
    if "time" in seq['headers'][0].lower():
        time = seq['data'][0]
        cols_i = 1
    # For the rest of the sequences just use the data as data
    for c in range(cols_i, cols_f):
        s = Timeseries( seq['headers'][c] )
        s.data = seq['data'][c]
        s.time = time
        seqs.append(s)
    raw_series.append(seqs)
    
# Access the information as `raw_series[0][1].name` (the name of the second column of the first sequence) 
print("Output included", len(raw_series), "sequences")
for s in range(len(raw_series)):
    print(f">>> Sequence {s} has headers: {[ h.name for h in raw_series[s] ]}")

In [ ]:
len(starts), len(ends)

## Plot the input and output

In [ ]:
import matplotlib.pyplot as plt

# Optioonally add an offset to better visualize the signals
OFFSET = 0

# make a deep copy to ensure that changes to the list will not affect the one obtained from the output
import copy 
series = copy.deepcopy(raw_series)
# Add to the sequence some reference signal
ts = Timeseries("Reference")
ts.data = din.data[0:]  + OFFSET
ts.name += " (offset)"
ts.time = din.time[0:]
series[0].append(ts)

# Add some time information based on knowledge of what was input and the expected output
series[0][0].time = ts.time
# series[0][1].time = ts.time
# series[0][2].time = ts.time

series[0][0].data = series[0][0].data + OFFSET
series[0][0].name += " (offset)"

# Generate subplot to show the results
plts = len(series)
plt_height = 5
fig, axs = plt.subplots(ncols=plts, figsize=(3*plt_height*plts, plt_height))

# Plot configurations
a = 0.5 # Alpha (opacity)

# Plot the desired signals
for p in range(plts):
    for s in range(len(series[p])):
        ax = axs if len(series) == 1 else axs[p] 
        if len(series[p][s].time) > 0:
            ax.plot(series[p][s].time, series[p][s].data, 
                    alpha = a)
        else:
            ax.plot(series[p][s].data, 
                    alpha = a)
    ax.legend([seq.name for seq in series[p]])
    ax.set_xlabel("Time (s)")
    ax.set_title(f"Filtered signals from {src_name}")


In [ ]:
error_count = 0
for i in range(len(series[0][0].data)):
    if(series[0][0].data[i] != series[0][1].data[i].astype(int)):
#         print(i, series[0][0].data[i], series[0][1].data[i].astype(int))
        error_count = error_count + 1
    
print(error_count, len(series[0][0].data))

In [ ]:
series[0][1].data[0:8]

## Performance and Energy

This will generate a report in `sw/riscv/build/energy_estim.csv`. 

Run the `estim.ipynb` notebook to visualize the results. 

In [ ]:
# Define some budgets that we should not cross
POWER_BUDGET_W      = 100e-6
LATENCY_BUDGET_S    = 1

xh.read_perf_cnt(perf_cnt);         # Read the performance counters
xh.estimate_performance(False);     # Estimate the performance 
xh.estimate_energy("HVT", False);   # Estimate the energy

rep_summary, rep_latency, rep_energy  = report() # Get the report

# Check that some contraints are satisfied
print("---------------------")
if rep_summary['Power_W']   < POWER_BUDGET_W:   print("✅ Power SUCCESS")   
else: print("❌ Power FAIL")
if rep_summary['Latency_s'] < LATENCY_BUDGET_S: print("✅ Latency SUCCESS") 
else: print("❌ Latency FAIL")

# Remove modules that will not be used in the design
pruned_energy = remove_module(rep_energy, "always-on peripheral subsystem" )
pruned_energy = remove_module(pruned_energy, "debug ao" )
pruned_energy = remove_module(pruned_energy, "spi" )
pruned_energy = remove_module(pruned_energy, "i2c" )

# Plot a breakdown of the energy consumption
plot_energy( pruned_energy, level=1 )

# In case you want to print the dictionaries containing the breakdown
if 0:
    import pprint
    pprint.pprint(pruned_energy)